In [12]:
import numpy as np
import pandas as pd
from small_text import LeastConfidence, PoolBasedActiveLearner, random_initialization_balanced, TextDataset
from small_text.integrations.transformers.classifiers.setfit import SetFitModelArguments
from small_text.integrations.transformers.classifiers.factories import SetFitClassificationFactory
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from synergy_dataset import Dataset, iter_datasets

from imblearn.over_sampling import RandomOverSampler
import torch

from sklearn.model_selection import train_test_split
import gc

In [13]:
dataset = Dataset('Radjenovic_2013')
dataset = dataset.to_frame()
# dataset = pd.read_csv('../datasets/synergy_dataset/Radjenovic_2013.csv')
dataset = dataset.dropna()

In [14]:
X = np.array(dataset['abstract'])
y = np.array(dataset['label_included'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [15]:
num_classes = 2
target_labels = [0, 1]
train = TextDataset.from_arrays(X_train, y_train, target_labels=target_labels)
test = TextDataset.from_arrays(X_test, y_test, target_labels=target_labels)

/home/junior/Mestrado Comp Ifes/Dissertação/small-text/.venv/lib/python3.8/site-packages/small_text/utils/annotations.py:67: ExperimentalWarning: The function from_arrays is experimental and maybe subject to change soon.
  warnings.warn(f'The {subject} {func_or_class.__name__} is experimental '
/home/junior/Mestrado Comp Ifes/Dissertação/small-text/.venv/lib/python3.8/site-packages/small_text/data/datasets.py:597: DeprecationWarning: The function get_flattened_unique_labels has been deprecated in 1.1.0 and will be removed in 2.0.0.
  encountered_labels = get_flattened_unique_labels(self)


In [16]:
sentence_transformer_model_name = 'sentence-transformers/paraphrase-mpnet-base-v2'
setfit_model_args = SetFitModelArguments(sentence_transformer_model_name)
clf_factory = SetFitClassificationFactory(setfit_model_args, num_classes, classification_kwargs={'device': 'cuda', 'max_seq_len': 128, 'mini_batch_size': 10})

In [17]:
query_strategy = LeastConfidence()
setfit_train_kwargs = {'show_progress_bar': False}
active_learner = PoolBasedActiveLearner(clf_factory, query_strategy, train, fit_kwargs={'setfit_train_kwargs': setfit_train_kwargs})

In [18]:
indices_initial = random_initialization_balanced(train.y, n_samples=20)
active_learner.initialize_data(indices_initial, train.y[indices_initial])

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
/home/junior/Mestrado Comp Ifes/Dissertação/small-text/.venv/lib/python3.8/site-packages/small_text/integrations/transformers/classifiers/setfit.py:260: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Map: 100%|██████████| 20/20 [00:00<00:00, 5701.11 examples/s]
/home/junior/Mestrado Comp Ifes/Dissertação/small-text/.venv/lib/python3.8/site-packages/small_text/integrations/transformers/classifiers/setfit.py:267: DeprecationWarning: `SetFitTrainer.train` does not accept keyword arguments anymore. Please provide training arguments via a `TrainingArguments` instance to the `SetFitTrainer` initialisation or the `SetFitTrainer.train` method.
  trainer.train(max_length=self.max_seq_len, **setfit_train_

{'embedding_loss': 0.16, 'learning_rate': 2.5e-06, 'epoch': 0.01}


{'embedding_loss': 0.0028, 'learning_rate': 8.333333333333334e-06, 'epoch': 0.62}



100%|██████████| 80/80 [00:37<00:00,  2.12it/s]


{'train_runtime': 37.7661, 'train_samples_per_second': 21.183, 'train_steps_per_second': 2.118, 'epoch': 1.0}


In [19]:
num_queries = 10

for i in range(num_queries):
    indices_queried = active_learner.query(num_samples=20)
    y = train.y[indices_queried]
    active_learner.update(y)
    gc.collect()
    torch.cuda.empty_cache()

    y_pred_train = active_learner.classifier.predict(train)
    y_pred_test = active_learner.classifier.predict(test)

    print(f'\nIteration {i+1}')
    print('Train accuracy: {:.2f}'.format(accuracy_score(train.y, y_pred_train)))
    print('Train precision: {:.2f}'.format(precision_score(train.y, y_pred_train, zero_division=np.nan)))
    print('Train recall: {:.2f}'.format(recall_score(train.y, y_pred_train, zero_division=np.nan)))
    print('Train F1 score: {:.2f}'.format(f1_score(train.y, y_pred_train)))
    print('\n')
    print('Test accuracy: {:.2f}'.format(accuracy_score(test.y, y_pred_test)))
    print('Test precision: {:.2f}'.format(precision_score(test.y, y_pred_test, zero_division=np.nan)))
    print('Test recall: {:.2f}'.format(recall_score(test.y, y_pred_test, zero_division=np.nan)))
    print('Test F1 score: {:.2f}'.format(f1_score(test.y, y_pred_test)))



/home/junior/Mestrado Comp Ifes/Dissertação/small-text/.venv/lib/python3.8/site-packages/small_text/data/datasets.py:597: DeprecationWarning: The function get_flattened_unique_labels has been deprecated in 1.1.0 and will be removed in 2.0.0.
  encountered_labels = get_flattened_unique_labels(self)
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
/home/junior/Mestrado Comp Ifes/Dissertação/small-text/.venv/lib/python3.8/site-packages/small_text/data/datasets.py:597: DeprecationWarning: The function get_flattened_unique_labels has been deprecated in 1.1.0 and will be removed in 2.0.0.
  encountered_labels = get_flattened_unique_labels(self)
/home/junior/Mestrado Comp Ifes/Dissertação/small-text/.venv/lib/python3.8/site-packages/small_text/integrations/transformers/classifiers/setfit.py:260: DeprecationWarning: `SetFitTrainer` has been deprecated and 

{'embedding_loss': 0.2762, 'learning_rate': 1.25e-06, 'epoch': 0.01}


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 7.92 GiB of which 47.88 MiB is free. Process 15954 has 3.02 GiB memory in use. Including non-PyTorch memory, this process has 4.49 GiB memory in use. Of the allocated memory 4.25 GiB is allocated by PyTorch, and 135.09 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)